### Achived Statements - indexing

Target:

 - us-east = 809,993,275
 - us-west = 809,993,275
 - archives = 3,239,979,516
 
Process:

 - bulk index the 3B archived statements using BAF
 
#### Principles

 - scale out the number of Nuxeo Worker nodes 
     - start with 5 worker nodes
     - ready to scale up to 8
     - bulk reindexing is configured with
         - 40 partitions
         - 5 threads/node to creating indexing commands
         - 4 threads/node to submit indexing commands


    es.baf.index.defaultPartitions=40
    es.baf.index.concurrency=5
    es.baf.bulkIndex.concurrency=4


- configure ES for bulk indexing (see [ES toolbox](../ToolBox/Elasticsearch.ipynb))
     - no replicas
     - refresh rate  
 - configure the archives MongoDB cluster use allow read from secondaries
     - add `?readPreference=nearest` to the connection url




Associated Datadog notebook: https://app.datadoghq.com/notebook/276275/index-archives-3b

#### Start the Bulk Indexing

In [ ]:
##!(INJECTOR="http://127.0.0.1:8080";\
curl -H 'Content-Type:application/json+nxrequest' \
  -H 'X-NXRepository:archives' \
  -X POST -d '{"params":{},"context":{}}' -u $NXUSER:$NXPWD \
  "$INJECTOR/nuxeo/api/v1/automation/Elasticsearch.BulkIndex")

In [ ]:
# check status
! cid="d1fc4c28-dafb-4c69-bf65-a7d8efdb2dc6"; \
  INJECTOR="http://127.0.0.1:8080"; \
  curl -H 'Content-Type:application/json+nxrequest' -H 'X-NXRepository:us-east' \
  -u $NXUSER:$NXPWD $INJECTOR/nuxeo/api/v1/bulk/$cid

#### Configure index for maximum speed

In [1]:
Index = "archives"

cmd = """
curl -X PUT $ES_SERVER/$Index/_settings -H "Content-Type: application/json" \
   -d '{"index" : { "number_of_replicas" : 0  , "refresh_interval" : -1 } }'

""".replace("$Index", Index)

print(cmd)

#!$cmd


curl -X PUT $ES_SERVER/archives/_settings -H "Content-Type: application/json"    -d '{"index" : { "number_of_replicas" : 0  , "refresh_interval" : -1 } }'


